---
bibliography: references.bib
---

# 가우스 소거법

연립 선형방정식을 푸는 데 도움이 되는 몇 가지 파이썬 함수를 사용하여, 가우스 소거법(Gaussian Elimination)에 대해서 알아보겠습니다. 가우스 소거법의 핵심은 해를 보존하면서 점차적으로 해에 더 쉽게 접근할 수 있도록 하는 **행 연산**이라는 일련의 단계를 적용하는 것 입니다. 기본 행 연산은 세 가지가 있습니다. 
1. 두 방정식의 위치 교환하기(interchange)
2. 방정식에 0이 아닌 숫자를 곱하기(scaling)
3. 특정 행을 자기 자신과 다른 행의 배수 배를 합한 것으로 교체(replacement)

## 예제 1:  Row operations and elimination

> [@linear_algebra_hanbit_2020], p64.

예를 들어 보겠습니다.

$$
\begin{matrix}
2x_1 + 2x_2 + 4x_3 & = & 18 \\
x_1 + 3x_2 + 2x_3 & = & 13 \\
3x_1 + x_2 + 3x_3 & = & 14 
\end{matrix}
$$

첫 번째 방정식과 마지막 방정식을 바꿀 수 있습니다.

$$
\begin{matrix}
3x_1 + x_2 + 3x_3 & = & 14 \\
x_1 + 3x_2 + 2x_3 & = & 13 \\
2x_1 + 2x_2 + 4x_3 & = & 18 \\
\end{matrix}
$$

또는 첫 번째 방정식에 $5$를 곱할 수 있습니다.

$$
\begin{matrix}
(2 \times 5)x_1 + (2 \times 5)x_2 + (4 \times 5)x_3 & = & 18 \\
x_1 + 3x_2 + 2x_3 & = & 13 \\
3x_1 + x_2 + 3x_3 & = & 14 
\end{matrix}
$$

또는 첫번째 방정식에 두번째 방정식의 $2$배를 곱하고 뺄 수도 있습니다.

$$
\begin{matrix}
x_1 - 4x_2 + x_3 & = & -8 \\
x_1 + 3x_2 + 2x_3 & = & 13 \\
3x_1 + x_2 + 3x_3 & = & 14 
\end{matrix}
$$

마지막 연산이 가장 중요한 이유는 방정식 중 하나에서 변수를 '제거'할 수 있기 때문입니다. 세 번째 방정식에는 더 이상 $x_2$ 항이 포함되어 있지 않다는 점에 유의하세요. 이것이 '가우스 소거법'의 핵심입니다.

계산을 위해 변수 이름과 "=" 기호를 생략하고 실제 숫자를 모두 배열로 정렬할 수 있습니다.

$$
\begin{matrix}
\left[ \begin{array}{ccc|c} 2 & 2 & 4 & 18 \\ 1 & 3 & 2 & 13 \\ 3 & 1 & 3 & 14 \end{array}\right]
\end{matrix}
$$

이제 이 값으로 NumPy 배열을 만들어 보겠습니다.  나중에 참조할 수 있도록 배열에 $\texttt{A}$라는 이름을 지정하겠습니다.

In [20]:
import numpy as np
import sympy as sp
A=np.array([[2,2,4,18],[1,3,2,13],[3,1,3,14]])

기본 행 연산을 수행할 수 있는 간단한 함수를 작성하도록 하겠습니다.

In [21]:
def row_swap(A,k,l):
    m = A.shape[0]
    n = A.shape[1]
    
    B = np.copy(A).astype('float64')
        
    for j in range(n):
        temp = B[k][j]
        B[k][j] = B[l][j]
        B[l][j] = temp
        
    return B

def row_scale(A,k,scale):
    m = A.shape[0]
    n = A.shape[1]
    
    B = np.copy(A).astype('float64')

    for j in range(n):
        B[k][j] *= scale
        
    return B

def row_add(A,k,l,scale):
    m = A.shape[0]
    n = A.shape[1]
    
    B = np.copy(A).astype('float64')
        
    for j in range(n):
        B[l][j] += B[k][j]*scale
        
    return B

이제 `row_swap`, `row_scale`, `row_add`라는 세 가지 새로운 함수가 생겼습니다. 이 함수를 사용해 어떤 결과가 나오는지 살펴봅시다.

In [22]:
B1 = row_scale(A,0,1/2)
B2 = row_add(A,1,0,-2)
B3 = row_swap(A,0,2)

In [23]:
print(A,'\n')
print(B1,'\n')
print(B2,'\n')
print(B3,'\n')

[[ 2  2  4 18]
 [ 1  3  2 13]
 [ 3  1  3 14]] 

[[ 1.  1.  2.  9.]
 [ 1.  3.  2. 13.]
 [ 3.  1.  3. 14.]] 

[[ 0. -4.  0. -8.]
 [ 1.  3.  2. 13.]
 [ 3.  1.  3. 14.]] 

[[ 3.  1.  3. 14.]
 [ 1.  3.  2. 13.]
 [ 2.  2.  4. 18.]] 



가우스 소거법의 목표는 행 연산을 수행하여 다음과 같은 구조의 행렬을 생성하는 것입니다.

$$
\begin{matrix}
\left[ \begin{array}{ccc|c} 1 & * & * & * \\ 0 & 1 & * & * \\ 0 & 0 & 1 & * \end{array}\right]
\end{matrix}
$$

*(여기서 * 기호는 0 또는 1이 될 수도 있고 아닐 수도 있는 다양한 미지의 값을 나타냅니다.)*.

행 연산을 수행하고 각 단계마다 새로운 이름을 붙인 배열로 진행 상황을 저장합니다.  예를 들어 $\texttt{A1}$, $\texttt{A2}$, $\texttt{A3}$ 등으로 이름을 지정할 수 있습니다. 이렇게 하면 진행 상황을 확인하거나 원하는 경우 돌아가서 코드를 변경할 수 있습니다.  

In [24]:
A1 = row_scale(A,0,1.0/2)
print(A1,'\n')
A2 = row_add(A1,0,1,-1)
print(A2,'\n')
A3 = row_add(A2,0,2,-3)
print(A3,'\n')
A4 = row_scale(A3,1,1.0/2)
print(A4,'\n')
A5 = row_add(A4,1,2,2)
print(A5,'\n')
A6 = row_add(A5,1,0,-1)
print(A6,'\n')
A7 = row_scale(A6,2,-(1.0/3))
print(A7,'\n')
A8 = row_add(A7,2,0,-2)
print(A8,'\n')

[[ 1.  1.  2.  9.]
 [ 1.  3.  2. 13.]
 [ 3.  1.  3. 14.]] 

[[ 1.  1.  2.  9.]
 [ 0.  2.  0.  4.]
 [ 3.  1.  3. 14.]] 

[[  1.   1.   2.   9.]
 [  0.   2.   0.   4.]
 [  0.  -2.  -3. -13.]] 

[[  1.   1.   2.   9.]
 [  0.   1.   0.   2.]
 [  0.  -2.  -3. -13.]] 

[[ 1.  1.  2.  9.]
 [ 0.  1.  0.  2.]
 [ 0.  0. -3. -9.]] 

[[ 1.  0.  2.  7.]
 [ 0.  1.  0.  2.]
 [ 0.  0. -3. -9.]] 

[[ 1.  0.  2.  7.]
 [ 0.  1.  0.  2.]
 [-0. -0.  1.  3.]] 

[[ 1.  0.  0.  1.]
 [ 0.  1.  0.  2.]
 [-0. -0.  1.  3.]] 



이제 행렬을 연립 선형 방정식 형태로 다시 표현하도록 하겠습니다.

$$
\begin{matrix}
x_1 \quad\quad & = & 1 \\
\quad x_2 \quad & = & 2 \\
\quad\quad x_3 & = & 3 \\
\end{matrix}
$$

## 예제 2: Finding pivots

마지막 예제의 코드는 배율을 계산하기 위해 나눈 배열의 항목 중 하나라도 0이 나타나면 실패합니다. 이러한 중요한 항목을 **피벗**이라고 하며, 행렬에서 해당 항목의 위치를 **피벗 위치**라고 합니다. 정의상 피벗은 0이 아니어야 합니다. 제거 단계에서 피벗 위치에 0이 발생하면 행의 순서를 바꿔서 0이 아닌 항목을 피벗 위치로 옮길 수 있습니다. 무작위 배열에서 작동하는 코드를 작성하기 전에 먼저 특정 배열에 대해 이 방법을 시도해 보겠습니다.

$$
\begin{matrix}
x_1 - x_2 + x_3 & = & 3\\
2x_1 - 2x_2 + 4x_3 & = & 8\\
3x_1 \quad\quad -9x_3 & = & 0 
\end{matrix}
$$


In [25]:
G=np.array([[1,-1,1,3],[2,-2,4,8],[3,0,-9,0]])
print(G)

[[ 1 -1  1  3]
 [ 2 -2  4  8]
 [ 3  0 -9  0]]


In [26]:
G1 = row_add(G,0,1,-2)
G2 = row_add(G1,0,2,-3)
print(G2)

[[  1.  -1.   1.   3.]
 [  0.   0.   2.   2.]
 [  0.   3. -12.  -9.]]


이제 중간 피벗 위치에 0이 있습니다.  제거를 계속하기 위해 중간 방정식과 마지막 방정식을 바꿀 수 있습니다.

In [27]:
G3 = row_swap(G2,1,2)
G4 = row_scale(G3,1,1./3)
G5 = row_scale(G4,2,1./2)
print(G5)

[[ 1. -1.  1.  3.]
 [ 0.  1. -4. -3.]
 [ 0.  0.  1.  1.]]


시스템을 익숙한 방정식 집합으로 다시 작성합니다.

$$
\begin{matrix}
x_1 - x_2 + x_3 & = & 3\\
x_2 - 4x_3 & = & -3\\
x_3 & = & 1 
\end{matrix}
$$

역치환을 적용하면 $x_2 = 1$과 $x_1=3$이 됩니다.

행을 바꾸는 것은 수식을 정리하는데 사용한다는 것에 주목할 필요가 있습니다. 동치항을 제거하여 해를 깔끔하게 나타낼 수 있습니다.

In [28]:
G3_alternative = row_scale(G2,1,1./2)
G4_alternative = row_scale(G3_alternative,2,1./3)
print(G4_alternative)

[[ 1. -1.  1.  3.]
 [ 0.  0.  1.  1.]
 [ 0.  1. -4. -3.]]


생성된 배열은 방정식의 순서는 물론 다르지만 동일한 단순화된 시스템을 나타냅니다.

$$
\begin{matrix}
x_1 - x_2 + x_3 & = & 3\\
x_3 & = & 1 \\
x_2 - 4x_3 & = & -3
\end{matrix}
$$

In [29]:
sp.Matrix(A).echelon_form()

Matrix([
[2, 2,   4,  18],
[0, 4,   0,   8],
[0, 0, -24, -72]])

In [30]:
sp.Matrix(A).rref()

(Matrix([
 [1, 0, 0, 1],
 [0, 1, 0, 2],
 [0, 0, 1, 3]]),
 (0, 1, 2))